In [9]:
import pandas as pd
from datetime import datetime, timezone
import folium
from add_utils import *

In [10]:
combined_mapping1= pd.read_excel("combined_mapping_results_09-20-23.xlsx")
combined_mapping2= pd.read_excel("combined_mapping_results_09-22-23.xlsx")
combined_mapping3= pd.read_excel("mapping_excel_files\\combined_mapping_results.xlsx")


In [11]:
combined_mapping3.head()

,chan,Latitude,Longitude,das_time_extrapolated
0,10884,37.273595,-121.830806,2023-10-02 10:02:31.018
1,10885,37.273590,-121.830799,2023-10-02 10:02:31.058
2,10886,37.273585,-121.830792,2023-10-02 10:02:31.099
3,10887,37.273580,-121.830786,2023-10-02 10:02:31.139
4,10888,37.273574,-121.830779,2023-10-02 10:02:31.180


In [12]:
#Loading the tap test excel file
local_folder = "C:/Users/varun/OneDrive - Stanford/Desktop/Stanford/3. Spring 25 Quarter/Independent Research- Geolocalization of Telecom Cables"
# local_folder = "C:/Users/varun/OneDrive - Stanford/DAS/2023-09-20"
tap_test_excel_path = f"{local_folder}/Tap-test-CCW.xlsx"
tap_test_df = pd.read_excel(tap_test_excel_path)
tap_test_df.columns
tap_test_df['Long'] = pd.to_numeric(tap_test_df['Long'], errors='coerce')

# Drop rows with invalid Lat or Long
tap_test_df = tap_test_df.dropna(subset=['Lat', 'Long'])
tap_test_df = parse_channel_column(tap_test_df)
tap_test_df.head()

,Unnamed: 0,Location,Lat,Long,Localized Channels,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Channel
0,1,Outside 1.1,37.336971,-121.886781,238-270,NaN,NaN,NaN,NaN,254.0
1,2,3rd/San Fernando,37.335225,-121.886894,661-678,NaN,NaN,NaN,NaN,669.0
2,3,3rd/San Carlos,37.332501,-121.884801,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3rd/William,37.329457,-121.882856,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1st/Reed,37.327266,-121.883827,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# ----------------------------
# 1) Create FINAL averaged mapping (one GPS per channel) + save to Excel
# ----------------------------
df_all = pd.concat(
    [combined_mapping1, combined_mapping2, combined_mapping3],
    ignore_index=True
)

Final_Combined_Mapping = (
    df_all
    .groupby("chan", as_index=False)
    .agg(
        Latitude=("Latitude", "mean"),
        Longitude=("Longitude", "mean"),
        n_samples=("Latitude", "count"),
    )
    .sort_values("chan")
)

# Save
Final_Combined_Mapping.to_excel("Final_Combined_Mapping.xlsx", index=False)

# ----------------------------
# 2) Downsample mapping: plot every 100th channel (by channel number)
# ----------------------------
Final_Combined_Mapping_100 = Final_Combined_Mapping[
    Final_Combined_Mapping["chan"] % 100 == 0
].copy()

# ----------------------------
# 3) Tap test points: plot ALL rows with valid Lat/Long
# ----------------------------
tap_pts = tap_test_df.copy()
tap_pts["Channel"] = pd.to_numeric(tap_pts["Channel"], errors="coerce")
tap_pts = tap_pts.dropna(subset=["Channel"])
tap_pts["Channel"] = tap_pts["Channel"].astype(int)
tap_pts = tap_pts.dropna(subset=["Lat", "Long"])

# ----------------------------
# 4) Folium plot: every 100th mapped channel + all tap test points
# ----------------------------
# Center map
if not Final_Combined_Mapping_100.empty:
    center_lat = float(Final_Combined_Mapping_100["Latitude"].median())
    center_lon = float(Final_Combined_Mapping_100["Longitude"].median())
else:
    center_lat = float(tap_pts["Lat"].median())
    center_lon = float(tap_pts["Long"].median())

m = folium.Map(location=[center_lat, center_lon], zoom_start=14, control_scale=True)

# Layer: every 100th channel from Final_Combined_Mapping
avg_layer = folium.FeatureGroup(name="Final mapping (chan % 100 == 0)", show=True)
for _, r in Final_Combined_Mapping_100.iterrows():
    folium.CircleMarker(
        location=[float(r["Latitude"]), float(r["Longitude"])],
        radius=3,
        weight=1,
        fill=True,
        fill_opacity=0.7,
        color="red",
        popup=folium.Popup(
            f"chan: {int(r['chan'])}<br>"
            f"n_samples: {int(r['n_samples'])}<br>"
            f"lat, lon: {r['Latitude']:.6f}, {r['Longitude']:.6f}",
            max_width=300
        ),
    ).add_to(avg_layer)
avg_layer.add_to(m)

# Layer: all tap test points
tap_layer = folium.FeatureGroup(name="Tap test points (all)", show=True)
for _, r in tap_pts.iterrows():
    loc_txt = r.get("Location", "")
    ch_txt = r.get("Channel", pd.NA)
    folium.Marker(
        location=[float(r["Lat"]), float(r["Long"])],
        icon=folium.Icon(icon="glyphicon-pushpin", prefix="glyphicon"),
        popup=folium.Popup(
            f"Location: {loc_txt}<br>"
            f"Channel: {'' if pd.isna(ch_txt) else int(ch_txt)}<br>"
            f"lat, lon: {float(r['Lat']):.6f}, {float(r['Long']):.6f}",
            max_width=350
        ),
    ).add_to(tap_layer)
tap_layer.add_to(m)

# Fit bounds to both layers
all_lat = pd.concat([Final_Combined_Mapping_100["Latitude"], tap_pts["Lat"]], ignore_index=True).dropna()
all_lon = pd.concat([Final_Combined_Mapping_100["Longitude"], tap_pts["Long"]], ignore_index=True).dropna()
if len(all_lat) and len(all_lon):
    m.fit_bounds([[float(all_lat.min()), float(all_lon.min())],
                  [float(all_lat.max()), float(all_lon.max())]])

folium.LayerControl(collapsed=False).add_to(m)

# Show in notebook
m

# Optional: save map
m.save("maps\\Final_Combined_Mapping_every100_with_tap.html")